In [1]:
from prepare_result import preapare_result

def read_txt(filename):
    l = []
    with open(filename, 'r') as file:
        tmp = ''
        for line in file:
            if not line.startswith('#'):
                parts = line.split('#')
                for part in parts:
                    part = part.strip()
                    if part:  # Check if the part is not an empty string
                        tmp = tmp + ' ' + part 

            if line.startswith('#'):
                l.append(tmp)
                tmp = ''

    l.append(tmp)
    return [i.strip() for i in l if len(i) > 0]

In [2]:
import numpy as np

def simplex_method(c, A, b, bounds):
    # Формируем таблицу
    A = np.array(A)
    m, n = A.shape
    tableau = np.zeros((m + 1, n + m + 1))
    tableau[:m, :n] = A
    tableau[:m, -1] = b
    tableau[-1, :n] = -c[0]
    tableau[-1, -1] = 1

    # Начальное базисное решение
    basic_variables = np.arange(n, n + m)
    non_basic_variables = np.arange(n)

    # Функция для проверки оптимальности
    def is_optimal(tableau):
        return all(tableau[-1, i] >= 0 for i in range(n + m))

    # Функция для выбора входящей переменной
    def entering_variable(tableau):
        return np.argmin(tableau[-1, :n])

    # Функция для выбора выходящей переменной
    def leaving_variable(tableau, entering_index):
        ratios = tableau[:-1, -1] / tableau[:-1, entering_index]
        leaving_index = np.argmin(ratios)
        return leaving_index

    # Функция для обновления таблицы
    def update_tableau(tableau, entering_index, leaving_index):
        pivot = tableau[leaving_index, entering_index]
        tableau[leaving_index, :] /= pivot
        for i in range(m + 1):
            if i != leaving_index:
                tableau[i, :] -= tableau[i, entering_index] * tableau[leaving_index, :]

    # Итерации симплекс-метода
    iterations = 0
    while not is_optimal(tableau):
        entering_index = entering_variable(tableau)
        leaving_index = leaving_variable(tableau, entering_index)
        update_tableau(tableau, entering_index, leaving_index)

        # Обновление базисных и небазисных переменных
        basic_variables[leaving_index] = entering_index
        non_basic_variables[entering_index] = leaving_index

        iterations += 1

    # Оптимальное решение и значение целевой функции
    optimal_solution = tableau[:-1, -1]
    optimal_value = -tableau[-1, -1]

    optimal_solution, optimal_value, iterations
    return preapare_result(c, A, b, bounds)

In [3]:
import numpy as np

def read_input_data(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    l = read_txt(file_path)

    c = list(map(float, l[0].split(' ')))
    
    L = list(map(float, l[1].split(' ')))
    L2 = []

    for i in range(0, len(L), 2):
        L2.append((L[i], L[i+1]))

    
    A = L2
    b = list(map(float, l[2].split(' ')))

    return c, A, b

def solve_linear_programming(file_path):
    c, A, b = read_input_data(file_path)

    bounds = [(0, None) for _ in c]  # Границы переменных (x_i >= 0)

    result = simplex_method(c, A=A, b=b, bounds=bounds)

    return result

def print_results(result):
    print("Оптимальные значения переменных:", result.x)
    print("Максимальное значение целевой функции:", -result.fun)

if __name__ == "__main__":
    input_file_path = "data/input_data.txt"
    result = solve_linear_programming(input_file_path)
    print_results(result)

Оптимальные значения переменных: [4.66666667 5.66666667]
Максимальное значение целевой функции: 40.333333333333336
